In [ ]:
%matplotlib ipympl

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout
import os
import textwrap

def read_file_contents(filename):
    with open(filename, 'r') as file:
        content = file.read()
        return content

def display_text(text, ax):
    # Clear previous text
    ax.clear()

    # Wrap the text
    wrapped_text = textwrap.fill(text, width=60)

    # Display the wrapped text
    ax.text(0.5, 0.5, wrapped_text, horizontalalignment='center', verticalalignment='center',
            fontsize=10, wrap=True, transform=ax.transAxes)

def on_click(event, ax, pos, ax_text):
    if event.artist is not None:
        node_id = event.artist.get_gid()
        file_path = f"dialogue_branch_{node_id}.txt"
        
        if os.path.exists(file_path):
            content = read_file_contents(file_path)
            annotation = f"Contents of {file_path}:\n{content}"
        else:
            annotation = f"No file associated with node {node_id}."
        
        # Update the text area with the new content
        display_text(annotation, ax_text)

        # Redraw node labels
        nx.draw_networkx_labels(G, pos, labels, font_size=8, ax=ax)

        # Redraw the figure
        fig.canvas.draw_idle()

def visualize_dialogue_tree(num_turns=10, num_responses=2):
    G = nx.DiGraph()
    labels = {}
    
    def add_nodes_and_edges(branch_id, depth):
        if depth >= num_turns:  # Change the condition here
            return
        
        node_id = f"{branch_id}"
        G.add_node(node_id)
        labels[node_id] = node_id
        
        for i in range(num_responses):
            child_id = f"{branch_id}-{i}"
            G.add_edge(node_id, child_id)
            add_nodes_and_edges(child_id, depth + 1)

    add_nodes_and_edges("0", 1)
    pos = nx.drawing.nx_agraph.graphviz_layout(G, prog="dot")

    fig, (ax, ax_text) = plt.subplots(2, 1, figsize=(20, 14), gridspec_kw={'height_ratios': [4, 1]})

    # Draw edges
    nx.draw_networkx_edges(G, pos, arrows=True, ax=ax)
    nx.draw_networkx_labels(G, pos, labels, font_size=8, ax=ax)

    # Draw nodes with picker enabled
    for node, position in pos.items():
        ax.scatter(position[0], position[1], s=1500, c="skyblue", gid=node, picker=10)

    fig.canvas.mpl_connect('pick_event', lambda event: on_click(event, ax=ax, pos=pos, ax_text=ax_text))

    # Set up the text area below the tree display
    ax_text.set_title('Selected Dialogue')
    ax_text.axis('off')
    display_text('Click on a node to display dialogue text.', ax_text)

    plt.show()

if __name__ == "__main__":
    visualize_dialogue_tree(num_turns=5, num_responses=2)
